네이버 영화 top 100 크롤링 순서
1. 네이버 영화 페이지별 링크 수집
2. 수집된 영화 링크에 접속하여 영화정보 크롤링
3. 데이터 분석에 필요한 csv파일로 저장

실시간 top100 시각화</br>
일간 top100 시각화</br>
주간 top100 시각화</br>
월간 top100 시각화

In [ ]:
# 링크 수집을 위한 라이브러리 설치
!pip install selenium

     |████████████████████████████████| 911kB 6.7MB/s 


In [ ]:
from bs4 import BeautifulSoup
import requests
from selenium import  webdriver as wd
from selenium.webdriver.common.keys import Keys
import time
import re
import pandas as pd


In [ ]:
# 웹드라이버 설치로 오류 방지할 수 있음
# install chromium, its driver, and selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
# set options to be headless, ..
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)


Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packag

분석할 데이터 모으기(크롤링)
* 네이버 영화 top100 페이지 별 링크 수집

In [ ]:
# https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_R&page= 실시간
# https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_D&page= 일간
# https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_W&page= 주간
# https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_M&page= 월간

def getPageLinksWantRange(startPageNo, LastPageNo) :
  links = [] # 100개의 영화 링크를 수집할 리스트 변수 선언
  
  for pageNo in range(startPageNo - 1, LastPageNo) :
    # 실시간 top100 영화 링크 수집
    url = 'https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_M&page=' + str(pageNo + 1)

    req = requests.get(url) # 페이지로 접속하기 // 페이지 소스를 싹 가져옴
    soup = BeautifulSoup(req.text, 'lxml') # 가져온 html을 파싱을 하기 위한 객체를 생성함
    movielinks = soup.select('div.lst_thum_wrap ul li a[href]') #<div class="lst_thum_wrap"> -> <ul> -> <li> -> <a href ="">


    for movielink in movielinks:
      link = str(movielink.get('href'))
      links.append('http://series.naver.com' + link) # 접속할 수 있는 전체 링크 형태로 변환해서 append함

  return links





In [ ]:
a = getPageLinksWantRange(1, 5)
print(a)

['http://series.naver.com/movie/detail.nhn?productNo=6119777', 'http://series.naver.com/movie/detail.nhn?productNo=6176165', 'http://series.naver.com/movie/detail.nhn?productNo=6158879', 'http://series.naver.com/movie/detail.nhn?productNo=6098871', 'http://series.naver.com/movie/detail.nhn?productNo=6023769', 'http://series.naver.com/movie/detail.nhn?productNo=6139377', 'http://series.naver.com/movie/detail.nhn?productNo=6139375', 'http://series.naver.com/movie/detail.nhn?productNo=6119778', 'http://series.naver.com/movie/detail.nhn?productNo=4356726', 'http://series.naver.com/movie/detail.nhn?productNo=6176518', 'http://series.naver.com/movie/detail.nhn?productNo=3293948', 'http://series.naver.com/movie/detail.nhn?productNo=6039515', 'http://series.naver.com/movie/detail.nhn?productNo=6176542', 'http://series.naver.com/movie/detail.nhn?productNo=1965132', 'http://series.naver.com/movie/detail.nhn?productNo=1808511', 'http://series.naver.com/movie/detail.nhn?productNo=6176166', 'http:/

네이버 영화제목, 평점, 장르, 줄거리 크롤링

In [ ]:
def getMovieDataFromNaverSeries(links):
  title_infos = [] # 제목
  content_infos = [] # 줄거리
  genre_infos = [] # 장르
  score_infos = [] # 평점
  date_infos = [] # 개봉일

  url2 = 'https://www.naver.com'

  driver = webdriver.Chrome('chromedriver', options=options)
  driver.get(url2)
  time.sleep(3.0) # 3초 sleep

  driver.find_element_by_css_selector('body').send_keys(Keys.CONTROL + 't')

  for link in links:
    print(link + '수집 중...........')
    driver.switch_to.window(driver.window_handles[-1]) # 새 창 활성화
    time.sleep(0.1)
    driver.get(link)
    time.sleep(0.1)
    driver.switch_to.window(driver.window_handles[0])
    time.sleep(3.0)

    html_source = driver.page_source # req.text 같은 동작
    html_soup = BeautifulSoup(html_source, 'lxml')

    # 청소년 관람불가 영화 크롤링을 할 경우 - 인증 - 수집에서 제외함
    flag = html_soup.text[0:10]

    newflag = ''.join(flag)
    newflag = newflag.replace('\n', '')

    if newflag == '네이버' :
      time.sleep(1.0)

      # 평점 수집
      score = driver.find_element_by_css_selector('div.score_area > em')
      score = float(score.text)
      score = int(score) # 정수형으로 변경하기
      score_infos.append(score)
      print(score)

      # 장르 수집
      genre = driver.find_element_by_css_selector('li.info_lst > ul > li:nth-child(4)').get_attribute('textContent')
      genre = genre.replace('장르', '')
      genre = genre.replace('\n', '')
      genre = genre.replace('\t', '')
      genre_infos.append(genre)

      # 제목, 줄거리

      try:
        movieInfoUrl =  driver.find_element_by_css_selector('li.info_lst > ul > li:nth-child(7) > a').get_attribute('href')
      except:
        movieInfoUrl =  driver.find_element_by_css_selector('li.info_lst > ul > li:nth-child(6) > a').get_attribute('href')

      # 영화 상세보기 페이지로 이동함
      movie_req = requests.get(movieInfoUrl)


      # 제목 수집
      movie_soup = BeautifulSoup(movie_req.text, 'lxml')
      title = movie_soup.head.find('meta', {'property' : 'og:title'}).get('content')
      title_infos.append(title)

      # 줄거리 수집
      contents_texts = movie_soup.select('div.story_area > p.con_tx')

      if len(contents_texts) == 0:
        content_infos.append('줄거리 오류')

      else:
        for contents in contents_texts :
          # 줄거리 데이터 클렌징 작업
          temp = contents.text
          temp = temp.replace('\r', '') # 줄바꿈 제거
          temp = temp.replace('\xa0', '') # 공백 제거
          content_infos.append(temp)

    elif newflag == '':
      print('청불 영화로 데이터를 수집하지 않습니다.')

  print('수집을 완료합니다..........')
  print(len(score_infos), len(genre_infos), len(content_infos))
  driver.close()

  movie_dic = {'제목' : title_infos, '평점' : score_infos, '장르' : genre_infos, '줄거리' : content_infos}

    # 딕셔너리 -> DataFrame
  movie_df = pd.DataFrame(movie_dic)

    # 수집된 정보 중에 중복 데이터 삭제
  movie_df2 = movie_df.drop_duplicates('줄거리', keep = 'first')

  return movie_df2 


      


CSV 파일로 저장

In [ ]:
def dftoCsv(movie_df, num):
  try:
    movie_df.to_csv(('movie_my_data_' + str(num) + '.csv'),
                    sep=',' , na_rep='NaN', encoding='euc-kr')
  except:
    print('Error')

크롤링 실행

In [ ]:
# 1. 크롤링할 링크 수집하기

links = getPageLinksWantRange(1, 5)

# 2. 영화 상세 정보 수집하기
movie_df2 = getMovieDataFromNaverSeries(links)

# 3. csv 파일 저장
dftoCsv(movie_df2, len(movie_df2)) # 그래도 한글 깨지면 encoding='utf-8-sig'


http://series.naver.com/movie/detail.nhn?productNo=6119777수집 중...........
7
http://series.naver.com/movie/detail.nhn?productNo=6176165수집 중...........
8
http://series.naver.com/movie/detail.nhn?productNo=6158879수집 중...........
9
http://series.naver.com/movie/detail.nhn?productNo=6098871수집 중...........
9
http://series.naver.com/movie/detail.nhn?productNo=6023769수집 중...........
9
http://series.naver.com/movie/detail.nhn?productNo=6139377수집 중...........
8
http://series.naver.com/movie/detail.nhn?productNo=6139375수집 중...........
9
http://series.naver.com/movie/detail.nhn?productNo=6119778수집 중...........
7
http://series.naver.com/movie/detail.nhn?productNo=4356726수집 중...........
9
http://series.naver.com/movie/detail.nhn?productNo=6176518수집 중...........
청불 영화로 데이터를 수집하지 않습니다.
http://series.naver.com/movie/detail.nhn?productNo=3293948수집 중...........
청불 영화로 데이터를 수집하지 않습니다.
http://series.naver.com/movie/detail.nhn?productNo=6039515수집 중...........
0
http://series.naver.com/movie/detail.nhn?produ

크롤링한 top100 영화 데이터 랭킹 시각화

라이브러리 임포트

In [7]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np


In [8]:
df = pd.read_csv('/content/movie_my_data_75.csv', encoding='cp949')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  75 non-null     int64 
 1   제목          75 non-null     object
 2   평점          75 non-null     int64 
 3   장르          75 non-null     object
 4   줄거리         75 non-null     object
dtypes: int64(2), object(3)
memory usage: 3.1+ KB


In [9]:
df.head()

,Unnamed: 0,제목,평점,장르,줄거리
0,0,콰이어트 플레이스,7,공포/스릴러,소리를 내는 순간 공격받는 극한의 상황 속에서 살아남기 위한 한 가족의 숨막히는 사...
1,1,극장판 귀멸의 칼날: 무한열차편,9,애니메이션,혈귀로 변해버린 여동생 ‘네즈코’를 인간으로 되돌릴 단서를 찾아 비밀조직 귀살대에 ...
2,2,킬러의 보디가드,8,액션,세상 제일 잘난 맛에 사는 섭외 1순위 앵그리 보디가드가지명수배 1순위 구강 액션 ...
3,3,크루엘라,9,드라마,처음부터 난 알았어. 내가 특별하단 걸그게 불편한 인간들도 있겠지만 모두의 비위를 ...
4,4,조커,8,드라마,고담시의 광대 아서 플렉은 코미디언을 꿈꾸는 남자.하지만 모두가 미쳐가는 코미디 같...


In [11]:
fig = px.scatter(df, x='제목', y='평점', color='장르')
fig.show()

In [12]:
df_score = df[df.평점 == 9]
df_score.tail()

,Unnamed: 0,제목,평점,장르,줄거리
54,65,밥정,9,다큐멘터리,"잔디, 잡초, 이끼, 나뭇가지.. 자연을 재료 삼아 요리를 만드는 방랑식객 임지호 ..."
58,69,소년시절의 너,9,액션,시험만 잘 치면 멋진 인생을 살 수 있다고 가르치는 세상에서기댈 곳 없이 세상에 내...
59,70,온워드: 단 하루의 기적,9,액션,마법이 사라진 세상에 살고 있는 취향과 성격 모두 정반대의 두 형제인철든 동생 ‘이...
63,74,기도의 힘,9,드라마,"성공 가도를 달리고 있는 남편 ‘토니’, 사랑스러운 딸 ‘대니엘’과 함께남부럽지 않..."
65,76,울지마 톤즈 2 : 슈크란 바바,9,다큐멘터리,"가난과 전쟁으로 아무런 희망조차 없었던 수단 톤즈에서사제이자 의사, 교육자, 음악가..."


9점 평점의 영화 중 가장 큰 비율을 차지하고 있는 장르는?

In [13]:
fig = px.pie(df_score, values = '평점', names='장르')
fig.show()

일간 top100 시각화

In [15]:
df2 = pd.read_csv('/content/movie_my_data_61.csv', encoding='euc-kr')
df2

,Unnamed: 0,제목,평점,장르,줄거리
0,0,극장판 귀멸의 칼날: 무한열차편,9,애니메이션,혈귀로 변해버린 여동생 ‘네즈코’를 인간으로 되돌릴 단서를 찾아 비밀조직 귀살대에 ...
1,1,콰이어트 플레이스,7,공포/스릴러,소리를 내는 순간 공격받는 극한의 상황 속에서 살아남기 위한 한 가족의 숨막히는 사...
2,2,킬러의 보디가드,8,액션,세상 제일 잘난 맛에 사는 섭외 1순위 앵그리 보디가드가지명수배 1순위 구강 액션 ...
3,3,크루엘라,9,드라마,처음부터 난 알았어. 내가 특별하단 걸그게 불편한 인간들도 있겠지만 모두의 비위를 ...
4,4,소울,9,애니메이션,뉴욕에서 음악 선생님으로 일하던 ‘조’는꿈에 그리던 최고의 밴드와 재즈 클럽에서 연...
...,...,...,...,...,...
56,70,비긴 어게인,9,코미디,싱어송라이터인 ‘그레타’(키이라 나이틀리)는 남자친구 ‘데이브’(애덤 리바인)가 메...
57,71,내가 널 사랑할 수 없는 10가지 이유,9,코미디,상냥하고 얼굴도 예쁜 비앙카(Bianca Stratford: 라리사 오레이닉 분)는...
58,72,내가 죽기를 바라는 자들,7,액션,화재 현장에서 세 명의 아이를 구하지 못한 죄책감과 트라우마로 감시탑에 배정된공수소...
59,73,애프터,7,드라마,첫 끌림대학 입학 후 첫 파티에 초대 받게 된 신입생 ‘테사’는진실게임을 하던 중 ...


In [19]:
fig = px.bar(df2, x='제목', y='평점', color='장르')
fig.show()

In [20]:
fig = px.pie(df2, values='평점', names='장르')
fig.show()